In [2]:
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from env import get_db_url
import wrangle


In [3]:
# exercises 2 question 1

In [4]:
#This function acquires the requisite zillow data from the Codeup SQL database and caches it locally it for future use in a csv document; once the data is accessed the function then returns it as a dataframe.
def get_zillow_data():
    filename = 'zillow_data.csv'

    if os.path.isfile(filename):
        return pd.read_csv(filename, index_col=0)
    else:
        df = pd.read_sql(
            '''
            SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips  FROM properties_2017 
            JOIN propertylandusetype USING (propertylandusetypeid) 
            WHERE propertylandusedesc = 'Single Family Residential'; 
            '''
            ,
            get_db_url('zillow')
        )

        df.to_csv(filename)

        return df

In [6]:
df = get_zillow_data()

In [7]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,parcelid
0,4.0,2.5,2490.0,691000.0,1969.0,7138.64,6059.0,14381521
1,4.0,2.0,1808.0,241131.0,1980.0,3589.81,6037.0,11108702
2,4.0,2.0,1878.0,94598.0,1965.0,1415.77,6037.0,10728687
3,4.0,2.0,1207.0,380736.0,1964.0,3869.62,6059.0,14376191
4,2.0,1.0,728.0,33497.0,1925.0,974.63,6037.0,12391744


In [8]:
df.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52442 entries, 0 to 52441
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bedroomcnt                    52442 non-null  float64
 1   bathroomcnt                   52442 non-null  float64
 2   calculatedfinishedsquarefeet  52360 non-null  float64
 3   taxvaluedollarcnt             52441 non-null  float64
 4   yearbuilt                     52326 non-null  float64
 5   taxamount                     52438 non-null  float64
 6   fips                          52442 non-null  float64
 7   parcelid                      52442 non-null  int64  
dtypes: float64(7), int64(1)
memory usage: 3.6 MB


In [ ]:
#2 handle nulls

In [13]:
# this function drops all the nulls in the data set
def handle_nulls(df):
    df = df.dropna()
    return df

# 3 final function 

makes bedroomcnt, calc finished sqr feet, bathroom cnt, taxvaluedollarcnt, year built, fips2 into int columns

In [15]:
def make_int(df):
    df = df.astype({'bedroomcnt':'int', 'calculatedfinishedsquarefeet':'int', 'taxvaluedollarcnt':'int', 'yearbuilt':'int','fips':'int'})
    return df

In [16]:
def fips2(fips):
  if fips == 'LA':
    return 6037
  elif fips == 'Orange':
    return 6059
  else:
    return 6111

In [17]:

def handle_outliers(df):
    df = df[df.bathroomcnt <= 6]

    df = df[df.bathroomcnt >= 1]
    
    df = df[df.bedroomcnt <= 6]

    df = df[df.bedroomcnt >= 1]

    df = df[df.taxvaluedollarcnt < 4_000_000]

    df = df[df.calculatedfinishedsquarefeet < 100000]

    return df

In [18]:
def wrangle_zillow():
    df= get_zillow_data()
    df = handle_nulls(df)
    df = make_int(df)
    df = handle_outliers(df)
    df['fips'].replace({6037 : 'LA' , 6059 : 'Orange' , 6111: 'Ventura'}, inplace = True)
    df['age'] = 2022 - df.yearbuilt
    df["fips2"] = df["fips"].apply(lambda fips: fips2(fips))
    return df 


In [21]:
df = wrangle_zillow()

In [22]:
df.shape

(51702, 10)